# Processing of RNA Illumina raw data
Note: Only iPSC processing shown, same thing for neurons and microglia

## Map with STAR

In [ ]:
cat ./IPSC_RAW_DATA/sample_names_ips.txt | while read -r first second;do
sbatch --mem=100g --cpus-per-task=2 --time=1-0 star_align.sh $HOME/RNA/illumina/STAR/index_hg38_mains/ \
./IPSC_RAW_DATA/"$second"_1.fastq.gz \
./IPSC_RAW_DATA/"$second"_2.fastq.gz \
./IPSC_PROCESSED_DATA/mapped/$first
done

## Run Salmon for quantification

In [ ]:
cat ./IPSC_RAW_DATA/sample_names_ips.txt | while read -r first second; do
sbatch --mem=50g --cpus-per-task=4 --time=1-0 --mail-type=END ./IPSC_PROCESSED_DATA/salmon/run_salmon.sh \
"$second" \
"$first"
done

## Create gene map

In [ ]:
Rscript --vanilla gencode_gene_map.r ./IPSC_PROCESSED_DATA/salmon/reference_files/gencode.v43.transcripts.fa.gz

## Run Tximport

In [ ]:
cat ../../IPSC_RAW_DATA/sample_names_ips.txt | while read -r first second; do
Rscript --vanilla tximport.r "gencode_txid_to_geneid.txt" "./$first/quant.sf" "./$first/$first.genes.csv" "$first" 
done

## Merge all tximport files

In [ ]:
# Generate file just with final file names in salmon folder
awk '{print $1}' ../../IPSC_RAW_DATA/sample_names_ips.txt > sample_names_ips_salmon.txt

In [ ]:
# Generate list to pass to R of file names
sh ./generate_file_names_R.sh

Copy paste in merging R script

In [ ]:
# Run merge R script
Rscript --vanilla merge_sample_salmon_genes.r

This gives you a final file: all_samples_salmon_genes_ips.csv